In [3]:
import numpy as np
import matplotlib.pyplot as plt
from presto.ppgplot import *
import pgplot_palette as pp
import scipy.ndimage as image
from astropy.coordinates import SkyCoord
from astropy import units as u


In [79]:
class Pointing():
    def __init__(self, srv_name, mjd, ra, dec, HAs, HAe):
        self.srv_name = srv_name
        self.mjd = mjd
        self.coord = SkyCoord(ra, dec, frame='icrs')
        self.dec = np.radians(dec)
        self.HA = np.array([HAs, HAe])
        
    def beam (self, gain=1.4):
        # Real gain
        g = 1.4938 * gain * np.sin(69.132*np.pi/180 - self.dec/2.)**2
        if self.dec < 28*np.pi/180:
            g = gain

        # Beamwidth    
        beamwidth = 18.048 / np.sin(69.132*np.pi/180 - self.dec/2.)
        if self.dec < 28*np.pi/180:
            beamwidth = 22

        # Compute min, max rotation of the beam    
        HA_rad = np.radians(self.HA)   
        beam_rot = np.degrees(np.arcsin(np.sin(self.dec)*np.sin(HA_rad)))
    
        #plot(HA, beam_rot)
        min_rot = np.min(beam_rot)
        max_rot = np.max(beam_rot)
        #print (min_rot, max_rot)        
        

class SPAN512():
    
    def __init__(self):
        
        self.srv_name, mjd, HAs, HAe, ra, dec = np.loadtxt("../AH/SPAN512.log", usecols = (0,2,5,6,14,15), unpack=True,dtype=str)     
        self.sc = SkyCoord(ra, dec, frame='icrs', unit=u.deg)     
        
        self.width = .4

    def __c2p(self, c):
        return (c.ra.hms[0]*3600 + c.ra.hms[1]*60 + c.ra.hms[2], c.dec.dms[0]*3600 + c.dec.dms[1]*60 + c.dec.dms[2])

    def check_src(self, c):
        if not isinstance(c, SkyCoord):
            print ("Object must be of Astropy class SkyCoord")
            return
        
        self.sep_array = self.sc.separation(c) < (2.5*self.width * u.deg)
        count = np.count_nonzero(self.sep_array)
        
    def plot_map(self, c, label=None, pgdev="/xw", label_SRV=None):
        xmin = self.__c2p(c)[0] - self.width * 4 * 60
        xmax = self.__c2p(c)[0] + self.width * 4 * 60
        ymin = self.__c2p(c)[1] - self.width * 60 * 60
        ymax = self.__c2p(c)[1] + self.width * 60 * 60
        #print(xmin, xmax, ymin, ymax)
        
        size = 512
        x = np.linspace(xmin, xmax, size)
        y = np.linspace(ymin, ymax, size)
        self.x, self.y = np.meshgrid(x, y)        
        self.m = np.zeros((size,size))
        #print (self.m.shape)
        
        self.check_src(c)
        
        xl = []
        yl = []
        ll = []
        
        self.list_pt = []
        # Loop over all SRV pointings
        for ii,cc in enumerate(self.sc):
            already_there = False
            
            # If SRV pointing close enough
            if self.sep_array[ii]:
                
                if c.separation(cc) < (0.4*self.width * u.deg):
                
                    x0, y0 = self.__c2p(cc)
                    xl.append(x0)
                    yl.append(y0)
                    ll.append(self.srv_name[ii])
                
                # Looking in uniq list of pointing
                if len(self.list_pt):
                    # Loop over uniq list of pointings
                    for c2 in self.list_pt:
                        if cc==c2:
                            already_there = True
                    if not already_there:
                        self.list_pt.append(cc)
                        self.m = np.maximum(self.m, self.add_beam(cc))
                
                else:                   
                    self.list_pt.append(cc)              
                    self.m = np.maximum(self.m, self.add_beam(cc))
        
        pgopen(pgdev)
        pgscf(2)
        pgsch(1.3)
        pgpap(0,1.)
        pgsvp(0.15,0.85,0.15,0.85)
        pgswin(xmin,xmax,ymin,ymax)
        pgtbox("ZXHOBCNSTI", 0.0, 2, "ZDBOCNSTI", 0.0, 2)

        Nlevels = 30
        pp.set_palette('viridis', Nlevels)
        pgscir(16,16+Nlevels)

        contours = np.arange(0.1,1.4,0.1)

        pgimag_s(self.m, 0, 1.4, xmin,ymin,xmax,ymax)
        pgcont_s(self.m,len(contours), contours, xmin,ymin,xmax,ymax)
        
        pgmtxt("L", 3.,0.5,0.5, "Declination (J2000)")
        pgmtxt("B", 3.,0.5,0.5, "Right Ascension (J2000)")
        
        if label:
            pgsci(2)
            X = (xmin+xmax)/2. + 0.05*(xmax-xmin)
            Y = (ymin+ymax)/2. + 0.05*(xmax-xmin)
            pgsch(1.2)
            pgptxt(X, Y, 0., 0., '\\fr\\fR'+label)
            pgsci(1)
            pgsch(1)
        
        if label_SRV:
            for x,y,l in zip(xl,yl,ll):
                pgsch(0.9)
                pgsci(0)
                pgptxt(x, y, 0., 0.5, l)
                pgsci(1)
                pgsch(1)
        
        pgsci(2)
        pgsch(3)
        pgpt(np.array([self.__c2p(c)[0]]), np.array([self.__c2p(c)[1]]), 2)
        pgsci(1)
        
        if not pgdev=="/xw":
            pgend()
        
        print("Telescope gain = %.2f K/Jy at the requested sky location (%s)"%(self.m[size//2,size//2], c.to_string('hmsdms')))        
    
    def add_beam (self, c):
        
        dec = c.dec.radian
        x0, y0 = self.__c2p(c)
        #print (c.to_string('hmsdms'), c.galactic)

        # Real gain
        gain = 1.4
        g = 1.4938 * gain * np.sin(69.132*np.pi/180 - dec/2.)**2
        if dec < 28*np.pi/180:
            g = gain

        # Beamwidth    
        beamwidth = 18.048 / np.sin(69.132*np.pi/180 - dec/2.)
        if dec < 28*np.pi/180:
            beamwidth = 22
        
        sigma_y = beamwidth/2.35 * 60  # in arcsec
        sigma_x = 4/2.35 * 4  / np.cos(dec) # in arcmin
        
        z = (1/(2*np.pi*sigma_x*sigma_y) * np.exp(-((self.x-x0)**2/(2*sigma_x**2) + (self.y-y0)**2/(2*sigma_y**2))))

        if np.max(z)>0:
            z = z/np.max(z)* g
        return z


In [80]:


s = SPAN512()
target = SkyCoord('01h11m21.87s', '+66d24m10.8s', frame='icrs')  
#target = SkyCoord('22h15m32.6884s', '+51d35m36.340s', frame='icrs')  
#target = SkyCoord('22h06m18.119s', '+61d51m58.10s', frame='icrs') 
#target = SkyCoord('20h55m10.306s', '+38d29m30.905s', frame='icrs') 
#target = SkyCoord('20h48m55.98s', '+49d51m53.021s', frame='icrs')
#target = SkyCoord('22h05m34.2014s', '+60d12m55.1408s', frame='icrs')
#target = SkyCoord('22h42m22.0s', '+63d50m41.0s', frame='icrs')
#target = SkyCoord('04h13m00.0s', '+58d00m00.0s', frame='icrs')
s.plot_map(target, label="PSR J2215+5135", pgdev = "/xw", label_SRV=True)

Telescope gain = 0.33 K/Jy at the requested sky location (01h11m21.87s +66d24m10.8s)


In [76]:
pgend()